In [1]:
root_path = "/tf/home/sergio/Tesis"

In [2]:
import sys
sys.path.append(root_path+"/TinyYOLOv3-Pedestrian-Detection")

from YOLOblocks import TinyYOLOv3,BasicBlock,PredictionLayer#,YOLOLossBasicBlock
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
#from tensorflow.python.tools import freeze_graph
#from skimage.io import imread,imshow
#from skimage.transform import resize 
import time
#from tensorflow.compat.v1.image import decode_image
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import os

In [3]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [4]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2070 SUPER, compute capability 7.5


## Dataset preparation

In [5]:
image_feature_description = {
    'bboxes': tf.io.FixedLenFeature([], tf.string),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'num_real_boxes':tf.io.FixedLenFeature([], tf.int64),
}

In [6]:
def get_iou_matrix_tf(box_arr1, box_arr2):
    
    box_arr1 = box_arr1 -tf.tile(box_arr1[:,:2],[1,2])
    #print(box_arr1)
    x11, y11, x12, y12 = tf.split(box_arr1, 4, axis=1)
    x21, y21, x22, y22 = tf.split(box_arr2, 4, axis=1)
    xA = tf.maximum(x11, tf.transpose(x21))
    yA = tf.maximum(y11, tf.transpose(y21))
    xB = tf.minimum(x12, tf.transpose(x22))
    yB = tf.minimum(y12, tf.transpose(y22))
    interArea = tf.maximum((xB - xA + 1e-9), 0) * tf.maximum((yB - yA + 1e-9), 0)
    boxAArea = (x12 - x11 + 1e-9) * (y12 - y11 + 1e-9)
    boxBArea = (x22 - x21 + 1e-9) * (y22 - y21 + 1e-9)
    iou = interArea / (boxAArea + tf.transpose(boxBArea) - interArea)
    return iou,tf.argmax(iou,axis=1)#[:,tf.newaxis]


def fill_yolo_output(boxes,grid_size,num_anchors,which_anchor_box,which_anchor_box_index):
    #print(boxes.shape)
    #noobj_mask = tf.ones((1,grid_size*grid_size*num_anchors))
    #print(noobj_mask.shape)
    
    x_min,y_min,x_max,y_max =tf.split(boxes,4,axis=1)

    #Transforma las coordenadas de (xmin,ymin,xmax,ymax) --> (xcenter,ycenter,width,height)
    width = x_max-x_min
    height = y_max-y_min
    x_global =x_min + tf.math.divide(x_max - x_min,2)
    y_global =y_min + tf.math.divide(y_max - y_min,2)
    
    
    x_min_anchor,y_min_anchor,x_max_anchor,y_max_anchor =tf.split(which_anchor_box,4,axis=1)
    
    width_anchor = x_max_anchor-x_min_anchor
    height_anchor = y_max_anchor-y_min_anchor
    x_global_anchor =x_min_anchor + tf.math.divide(x_max_anchor - x_min_anchor,2)
    y_global_anchor =y_min_anchor + tf.math.divide(y_max_anchor - y_min_anchor,2)   

    
    #print("el x original",x_global)
    #print("el y original",y_global)
    #print("el w original",width)
    #print("el h original",height)
    
    #porción de la imagen que hay en cada celda
    pixel_per_grid = tf.math.divide(1.,grid_size)
    #print(pixel_per_grid)
    
    #Obtenemos la coordenada de la celda donde están los boundingboxes
    offset_grid_x = x_global//pixel_per_grid
    offset_grid_y = y_global//pixel_per_grid
    
    #Obtenemos el el centro locacon referencia  al celda encontrada previamente
    x_local =tf.math.floormod(x_global,pixel_per_grid)
    y_local =tf.math.floormod(y_global,pixel_per_grid)
    #print(x_local,y_local)
    
    #Valores tx e ty del groudtruth
    tx = tf.math.log(x_local + 1e-07/(1-x_local))
    ty = tf.math.log(y_local+1e-07/(1-y_local))
    tw = tf.math.log(tf.math.divide(width+1e-07,width_anchor))
    th = tf.math.log(tf.math.divide(height+1e-07,height_anchor))
    tobj_mask = tf.ones_like(tx)
    tobj = tf.concat([tobj_mask,tobj_mask],axis=0)
    
    #tnoobj = tf.zeros_like(tx)    
    #tobj = tf.ones((grid_size*grid_size*num_anchors,1))
    #tnoobj = tf.zeros((grid_size*grid_size*num_anchors,1))
    #print("Lo que la red debe predecir",tx.numpy(),ty.numpy(),tw.numpy(),th.numpy())
    #x_global = (offset_grid_x * pixel_per_grid) + tf.math.sigmoid(tx)
    #y_global = (offset_grid_y * pixel_per_grid) + tf.math.sigmoid(ty)
    #w = width_anchor*tf.math.exp(tw)
    #h = height_anchor*tf.math.exp(th)
    #print("obtnemos el x_real",x_global)
    #print("obtenemos el y_real",y_global)
    #print("obtenemos el w real",w)
    #print("obtenemos el h real",h)
    
    #anchor_boxes_per_output = num_anchors//2

    #Residuo indica cual de los 3 anchor boxes de la coordenada es la que llevara el 1
    #Coord representa la coordenada del grid
    
    residuo = tf.math.floormod(which_anchor_box_index,num_anchors)[:,tf.newaxis]
    coord = tf.cast(num_anchors*(offset_grid_y*grid_size + offset_grid_x),dtype=tf.int64)
    
    coord_objectness = tf.cast(2*(offset_grid_y*grid_size + offset_grid_x),dtype=tf.int64)
    coord_objectness2 = coord_objectness+1
    coord_objectess_global = tf.concat([coord_objectness,coord_objectness2],axis=0)
    
    output_position = residuo+coord
    print("tipo de aoutput_positivon",output_position)
    
    print(output_position)
    
    dense_shape = grid_size*grid_size*num_anchors
    print(dense_shape)
    tx_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tx[:,0], dense_shape=[dense_shape]))
    ty_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=ty[:,0], dense_shape=[dense_shape]))
    tw_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tw[:,0], dense_shape=[dense_shape]))
    th_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=th[:,0], dense_shape=[dense_shape]))
    obj_mask = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tobj_mask[:,0], dense_shape=[dense_shape]))
    objectness_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=coord_objectess_global, values=tobj[:,0], dense_shape=[dense_shape]))
    #noobj_mask = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tnoobj[:,0], dense_shape=[dense_shape]))
    #obj_mask =tx_vector=ty_vector=tw_vector=th_vector = tf.zeros((1,grid_size*grid_size*num_anchors))
    
    tx_vector_dense = tf.sparse.to_dense(tx_vector, default_value=0, validate_indices=False, name="Dense_tx")
    ty_vector_dense = tf.sparse.to_dense(ty_vector, default_value=0, validate_indices=False, name="Dense_ty")
    tw_vector_dense = tf.sparse.to_dense(tw_vector, default_value=0, validate_indices=False, name="Dense_tw")
    th_vector_dense = tf.sparse.to_dense(th_vector, default_value=0, validate_indices=False, name="Dense_th")
    obj_mask_dense =  tf.sparse.to_dense(obj_mask, default_value=0, validate_indices=False, name="Dense_obj")
    #noobj_mask_dense = 1-obj_mask_dense
    objectness_vector_dense =  tf.sparse.to_dense(objectness_vector, default_value=0, validate_indices=False)
    
    #noobj_mask_dense= tf.sparse.to_dense(noobj_mask, default_value=1, validate_indices=False, name="Dense_noobj")
    ##print(tx_vector.to_dense)
    #print(tf.sparse.to_dense(tx_vector, default_value=0, validate_indices=True, name=None)
    #tx_vector=tx_vector[[3,2],]
    #tx_vector[output_position[:,0]] = tx
    #print("coordenada de la salida:",output_position)
    
    #return ((tx_vector_dense,ty_vector_dense,obj_mask_dense),(tw_vector_dense,th_vector_dense,obj_mask_dense),(objectness),(objectness))
    
    return tx_vector_dense,ty_vector_dense,tw_vector_dense,th_vector_dense,obj_mask_dense,objectness_vector_dense

def build_targets(image,image_bboxes,num_real_boxes,anchor_boxes):
    
    images_bboxes_original = image_bboxes
    #Obtenemos los boduing boxes que son reales
    image_bboxes = image_bboxes[:num_real_boxes,:]
    #print("Bouding boxes de la imagen",image_bboxes)
    #Obteneos  la matriz de IoU , y el índice del anchor box que dió mejor resultado
    
    #Nprmalizamos con respecto al tamaño de la imagen y obtenemos la Iou con los anchor boxes
    image_bboxes = tf.math.divide(image_bboxes,416)
    iou_matrix,which_anchor_box_index = get_iou_matrix_tf(image_bboxes,anchor_boxes)
    
    print(which_anchor_box_index)

    anchor_boxes_per_output = len(anchor_boxes)//2
    #Indices de los bouding boxes que irian en cada salida, index_best_ yolo nos dice que bouding boxes de la imagen van a la salida YOLO1,
    #porque su mejor IoU fue con los len(anchor_boxes)//2 anchor boxes mas grandes
    index_best_yolo1 = tf.where(which_anchor_box_index>=anchor_boxes_per_output)[:,0]
    index_best_yolo2 = tf.where(which_anchor_box_index<anchor_boxes_per_output)[:,0]
    index_best_anchor_yolo1 = tf.gather(which_anchor_box_index,index_best_yolo1,axis=0)
    index_best_anchor_yolo2 = tf.gather(which_anchor_box_index,index_best_yolo2,axis=0)
    
    print(index_best_yolo1)
    print(index_best_anchor_yolo1)

    print(index_best_yolo2)
    print(index_best_anchor_yolo2)

    
    best_bboxes_yolo1 = tf.gather(image_bboxes,index_best_yolo1,axis =0)
    best_anchors_yolo1 = tf.gather(anchor_boxes,index_best_anchor_yolo1, axis =0) #LOs dos anchor boxes grandes corrsponden a YOLO1
    best_bboxes_yolo2 = tf.gather(image_bboxes,index_best_yolo2,axis =0)
    best_anchors_yolo2 = tf.gather(anchor_boxes,index_best_anchor_yolo2, axis =0) #Los dos anchor boxes pequeños corresponden a YOLO2
    
    
    if best_anchors_yolo1.shape[0] !=0:
        tx_vector_yolo1,ty_vector_yolo1,tw_vector_yolo1,th_vector_yolo1,obj_mask_yolo1,obj_vector_yolo1= fill_yolo_output(best_bboxes_yolo1,13,anchor_boxes_per_output,best_anchors_yolo1,index_best_anchor_yolo1)
    else:
        tx_vector_yolo1=ty_vector_yolo1=tw_vector_yolo1=th_vector_yolo1=obj_mask_yolo1= obj_vector_yolo1=tf.zeros((1,grid_size*grid_size*num_anchors))
        #noobj_mask_yolo1 = tf.ones((1,13*13*num_anchors))
    
    if best_anchors_yolo2.shape[0] != 0:
        tx_vector_yolo2,ty_vector_yolo2,tw_vector_yolo2,th_vector_yolo2,obj_mask_yolo2,obj_vector_yolo2 = fill_yolo_output(best_bboxes_yolo2,26,anchor_boxes_per_output,best_anchors_yolo2,index_best_anchor_yolo2)
    else:
        tx_vector_yolo2=ty_vector_yolo2=tw_vector_yolo2=th_vector_yolo2=obj_mask_yolo2 = obj_vector_yolo2=tf.zeros((1,grid_size*grid_size*num_anchors))
        #noobj_mask_yolo2 = tf.ones((1,26*26*num_anchors))
        
    tx_vector = tf.concat([tx_vector_yolo1,tx_vector_yolo2],axis=0)[:,tf.newaxis]
    ty_vector = tf.concat([ty_vector_yolo1,ty_vector_yolo2],axis=0)[:,tf.newaxis]
    tw_vector = tf.concat([tw_vector_yolo1,tw_vector_yolo2],axis=0)[:,tf.newaxis]
    th_vector = tf.concat([th_vector_yolo1,th_vector_yolo2],axis=0)[:,tf.newaxis]
    obj_mask = tf.concat([obj_mask_yolo1,obj_mask_yolo2],axis=0)[:,tf.newaxis]
    #noobj_mask = tf.concat([noobj_mask_yolo1,noobj_mask_yolo2],axis=0)[:,tf.newaxis]
    obj_vector = tf.concat([obj_vector_yolo1,obj_vector_yolo2],axis=0)[:,tf.newaxis]
    
    #output = tf.concat([tx_vector,ty_vector,tw_vector,th_vector,obj_mask,noobj_mask,obj_vector],axis=1)
    #images_bboxes_original
    #return image,output
    #Vamos a regresar obj mask que es 1 cuando hay objeto en grid y el anchor box especifico
    return tf.cast(image,tf.float32)/255,(tf.concat([tx_vector,ty_vector,obj_mask],axis=1),tf.concat([tw_vector,th_vector,obj_mask],axis=1),(obj_mask),(obj_mask))

def imgaug_data_augmentation(image,bboxes,num_real_boxes):
    im_shape = image.shape
    bbs = BoundingBoxesOnImage.from_xyxy_array(bboxes*416, shape=(416,416))
    
    policy = np.random.randint(5)
    
    #policy = 2
    if policy == 0:
        
        p = np.random.random()
        if p<=0.6:
            aug = iaa.TranslateX(px=(-60, 60),cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
    
        p = np.random.random()
        if p<=0.8:
            aug = iaa.HistogramEqualization()
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
    
    elif policy==1:
        
        p=np.random.random()
        if p<=0.2:
            aug = iaa.TranslateY(px=(int(-0.18*416), int(0.18*416)),cval=128)
            for i in bbs.to_xyxy_array(np.int32)[:num_real_boxes,:]:
                bbox = image[i[1]:i[3],i[0]:i[2]]
                bbox_augmented = aug(image=bbox)
                image[i[1]:i[3],i[0]:i[2]] = bbox_augmented
        
        p=np.random.random()
        if p<=0.8:
            square_size = np.random.randint(48)
            aug = iaa.Cutout(nb_iterations=1, size=square_size/416, squared=True)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
    elif policy==2:
        p=np.random.random()
        if p<=1:
            aug = iaa.ShearY(shear=(int(-0.06*416), int(0.06*416)), order=1, cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
        p=np.random.random()
        if p<=0.6:
            aug = iaa.TranslateY(px=(int(-0.18*416), int(0.18*416)),cval=128)
            for i in bbs.to_xyxy_array(np.int32)[:num_real_boxes,:]:
                bbox = image[i[1]:i[3],i[0]:i[2]]
                bbox_augmented = aug(image=bbox)
                image[i[1]:i[3],i[0]:i[2]] = bbox_augmented
            
    elif policy==3:
        p=np.random.random()
        if p<=0.6:    
            aug = iaa.Rotate(rotate=(-30, 30), order=1, cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs_aug.remove_out_of_image().clip_out_of_image()
        
        p=np.random.random()
        if p<=1:
            aug = iaa.MultiplySaturation((0.54, 1.54))
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
    bbs.remove_out_of_image()
    
    return image,np.clip(bbs.to_xyxy_array(np.float32),1,415),num_real_boxes
    
    
def preprocessing(example_proto):
    image_features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.image.decode_jpeg(image_features['image_raw'],channels = 3)
    image = tf.cast(tf.image.resize(image,size=(416,416)), tf.uint8)
    bboxes =  tf.io.parse_tensor(image_features['bboxes'], out_type=tf.float32)
    
    num_real_boxes = image_features['num_real_boxes']
    return image,bboxes,num_real_boxes

def preprocessing_validation_set(example_proto):
    image_features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.image.decode_jpeg(image_features['image_raw'],channels = 3)
    image = tf.cast(tf.image.resize(image,size=(416,416)), tf.uint8)
    bboxes =  tf.io.parse_tensor(image_features['bboxes'], out_type=tf.float32)
    bboxes = tf.clip_by_value(bboxes*416,1,415)
    
    num_real_boxes = image_features['num_real_boxes']
    return image,bboxes,tf.cast(num_real_boxes,tf.int64)
    
@tf.function(input_signature=[tf.TensorSpec((416,416,3), tf.uint8),tf.TensorSpec((None,4), tf.float32),tf.TensorSpec((), tf.int64)]) 
def tf_numpy_albumentations_real(image,bboxes,num_real_boxes):
    
    boxes_shape = bboxes.shape
    im_shape = image.shape

    image,bboxes,num_real_boxes = tf.numpy_function(imgaug_data_augmentation,[image,bboxes,num_real_boxes],Tout =[tf.uint8,tf.float32,tf.int64])
 
    image.set_shape(im_shape)
    bboxes.set_shape(boxes_shape)
    print("Imagen data type",image.dtype)
    print("Bboxes data type",bboxes.dtype)
    print("num_real_boxes",num_real_boxes.dtype)

    return image,bboxes,num_real_boxes

### Tranfer Learning without Data Augmentation

Comentamos la operacion de map en la función de tf_numpy_albumentations y ademas usamos la función preprocessing_validation, esto nos permitirá leer los datos sin data augmentatio, además usaremos la la opcion de mode="transfer" para hacer No entrenables todas las capas menos las de detección

In [7]:
#USANDO TF.IMAGE MODULE
anchors =tf.constant(np.array([[0,0,0.026,0.062],[0,0,0.067,0.183],[0,0,0.128,0.323],[0,0,0.343,0.650]]),dtype=tf.float32)
#anchors =tf.constant(np.array([[0,0,0.02078,0.049],[0,0,0.0426,0.128],[0,0,0.08523,0.19356],[0,0,0.1506,0.4163],[0,0,0.27835,0.58651],[0,0,0.5632,0.78614]]),dtype=tf.float32)

os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")
filenames = os. listdir()
raw_image_dataset = tf.data.TFRecordDataset(filenames)

os.chdir("/home/sergio/Documents/pedestrian_dataset_val_tfr_fixed")
filenames = os. listdir()
raw_image_dataset_val =tf.data.TFRecordDataset(filenames)

os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")
#.shuffle(70000)
train_dataset = raw_image_dataset.map(preprocessing_validation_set,num_parallel_calls=8)
#train_dataset = train_dataset.map(tf_numpy_albumentations_real,num_parallel_calls=8)
train_dataset = train_dataset.map(lambda x,y,z:build_targets(x,y,z,anchors),num_parallel_calls=8)
train_dataset = train_dataset.batch(16)

val_dataset = raw_image_dataset_val.map(preprocessing_validation_set,num_parallel_calls=8)
val_dataset = val_dataset.map(lambda x,y,z:build_targets(x,y,z,anchors),num_parallel_calls=8)
val_dataset = val_dataset.batch(16)

FileNotFoundError: [Errno 2] No such file or directory: '/home/sergio/Documents/pedestrian_dataset_train_tfr'

### Loss Function

In [5]:
from tensorflow.keras.losses import Loss,BinaryCrossentropy,MeanSquaredError,MeanSquaredLogarithmicError

def loss_xy(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y

def loss_wh(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h

'''

def loss_xy(y_true,y_pred):
    
    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)

    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tx_true-tx_pred))  
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y

def loss_wh(y_true,y_pred):

    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tw_true-tw_pred))

    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h


def loss_xy(y_true,y_pred):
    
    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)

    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tx_true-tx_pred))  
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y

def loss_wh(y_true,y_pred):

    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tw_true-tw_pred))

    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tf.math.exp(tw_true),tf.math.exp(tw_pred))[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tf.math.exp(th_true),tf.math.exp(th_pred))[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h
'''

def loss_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   
    
    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_obj

def loss_no_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    
    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_noobj
'''
def loss_bce_objectness(y_true,y_pred):
    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   
    
    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*tf.math.log(y_pred+1e-7)[:,tf.newaxis],axis=1))
    
    return loss_obj

def loss_bce_no_objectness(y_true,y_pred):
    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*tf.math.log(1-y_pred+1e-7)[:,tf.newaxis],axis=1))
    
    return loss_noobj
'''

'\ndef loss_bce_objectness(y_true,y_pred):\n    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)\n    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)\n    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   \n    \n    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*tf.math.log(y_pred+1e-7)[:,tf.newaxis],axis=1))\n    \n    return loss_obj\n\ndef loss_bce_no_objectness(y_true,y_pred):\n    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)\n    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*tf.math.log(1-y_pred+1e-7)[:,tf.newaxis],axis=1))\n    \n    return loss_noobj\n'

## Training with Model.fit()

In [6]:
anchors =tf.constant(np.array([[0.026,0.062],[0.067,0.183],[0.128,0.323],[0.343,0.650]]),dtype=tf.float32)
#anchors =tf.constant(np.array([[0,0,0.02078,0.049],[0,0,0.0426,0.128],[0,0,0.08523,0.19356],[0,0,0.1506,0.4163],[0,0,0.27835,0.58651],[0,0,0.5632,0.78614]]),dtype=tf.float32)

model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "transfer")
model.build(batch_input_shape=(None,416,416,3))
print(model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))

Modo entrenamiento
Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (4718592,)
CONV SHAPE (1024, 512, 3, 3)
Pesos de la convolucion (262144,)
CONV SHAPE (256, 1024, 1, 1)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (130560,)
CONV SHAPE (255, 512, 1, 1)
Pesos de la convolucion (32768,)
CONV SHAPE (128, 256, 1, 1)
Pesos de la convolucion (884736,)
CONV SHAPE (256, 384, 3, 3)
Pesos de la convolucion (65280,)
CONV SHAPE (255, 256, 1, 1)
8858734


In [7]:
for l in model.layers:
    print(l.name, l.trainable)#,l.weights[0].shape)
    pass

BasicBlock1 False
BasicBlock2 False
BasicBlock3 False
BasicBlock4 False
BasicBlock5 False
BasicBlock6 False
BasicBlock7 False
BasicBlock8 False
BasicBlock9 False
FinalBlock1 True
BasicBlock11 False
BasicBlock12 False
FinalBlock2 True
Concatenate True
Upsampling True
Prediction1 True
Prediction2 True
Concatenate_BBOX True


In [8]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=1e-4)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(0.5),Recall(0.5),TrueNegatives(0.5),TruePositives(0.5),FalseNegatives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[5,5,2,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

Entrenamiento tipo transfer learning con lr = 0.0001 y por 20 epocas, la función de costo será con mse normal y sin Data Augmentation y ahora con 4 anchors totales, no con 6

In [9]:
history = model.fit(train_dataset, epochs=20,validation_data=val_dataset)

Modo entrenamiento
Epoch 1/20
Modo entrenamiento
Modo entrenamiento
   4607/Unknown - 189s 41ms/step - loss: 154.4877 - output_1_loss: 14.9517 - output_2_loss: 2.7908 - output_3_loss: 0.1695 - output_4_loss: 65.4361 - output_3_precision: 0.0065 - output_3_recall: 0.0264 - output_3_true_negatives: 123189080.0000 - output_3_true_positives: 7129.0000 - output_3_false_negatives: 262435.0000 - output_3_false_positives: 1091003.0000 175s 41ms/st - 182s 41ms/step - loss: 158.0786 - outp - 188s 41ms/step - loss: 154.9713 - output_1_loss: 14.9899 - output_2_loss: 2.7981 - output_3_loss: 0.1700 - output_4_loss: 65.6910 - output_3_precision: 0.0064 - output_3_recall: 0.0262 - output_3_true_negatives: 122538368.0000 - output_3_true_positives: 7031.0000 - output_3_false_negatives: 26Modo entrenamiento
4607/4607 [==============================] - 196s 43ms/step - loss: 154.4877 - output_1_loss: 14.9517 - output_2_loss: 2.7908 - output_3_loss: 0.1695 - output_4_loss: 65.4361 - output_3_precision: 0.0

In [10]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_transfer_5521_20_epoch_nadam_0dot00001_mse_2anchors.json', 'w') as fp:
    json.dump(str(history.history), fp)

In [11]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_5521_20_epoch_nadam_0dot00001_mse_2anchors')

ME FALTA PROBAR 2 ANCHORS CON DATA AUGMENTATIO

### Transfer Learning with Data Augmentation 20 epochs

Entrenamiento tipo transfer learning con lr = 0.00001 y por 10 epocas, la función de costo será con mse normal y sin Data Augmentation

Usaremos la opcion de mode="transfer" para hacer No entrenables todas las capas menos las de detección

In [7]:
#USANDO TF.IMAGE MODULE
#anchors =tf.constant(np.array([[0,0,0.015,0.037],[0,0,0.043,0.104],[0,0,0.11,0.278],[0,0,0.351,0.66]]),dtype=tf.float32)
#anchors =tf.constant(np.array([[0,0,0.026,0.062],[0,0,0.067,0.183],[0,0,0.128,0.323],[0,0,0.343,0.650]]),dtype=tf.float32)
anchors =tf.constant(np.array([[0,0,0.02078,0.049],[0,0,0.0426,0.128],[0,0,0.08523,0.19356],[0,0,0.1506,0.4163],[0,0,0.27835,0.58651],[0,0,0.5632,0.78614]]),dtype=tf.float32)

os.chdir(root_path+"/pedestrian_dataset_train_tfr")
filenames = os. listdir()
raw_image_dataset = tf.data.TFRecordDataset(filenames)

os.chdir(root_path+"/pedestrian_dataset_val_tfr_fixed")
filenames = os. listdir()
raw_image_dataset_val =tf.data.TFRecordDataset(filenames)

os.chdir(root_path+"/pedestrian_dataset_train_tfr")

train_dataset = raw_image_dataset.map(preprocessing,num_parallel_calls=8)
train_dataset = train_dataset.map(tf_numpy_albumentations_real,num_parallel_calls=8)
train_dataset = train_dataset.map(lambda x,y,z:build_targets(x,y,z,anchors),num_parallel_calls=8)
train_dataset = train_dataset.batch(16)

val_dataset = raw_image_dataset_val.map(preprocessing_validation_set,num_parallel_calls=8)
val_dataset = val_dataset.map(lambda x,y,z:build_targets(x,y,z,anchors),num_parallel_calls=8)
val_dataset = val_dataset.batch(16)

Imagen data type <dtype: 'uint8'>
Bboxes data type <dtype: 'float32'>
num_real_boxes <dtype: 'int64'>
Tensor("ArgMax:0", shape=(None,), dtype=int64)
Tensor("strided_slice_2:0", shape=(None,), dtype=int64)
Tensor("GatherV2:0", shape=(None,), dtype=int64)
Tensor("strided_slice_3:0", shape=(None,), dtype=int64)
Tensor("GatherV2_1:0", shape=(None,), dtype=int64)
tipo de aoutput_positivon Tensor("add_18:0", shape=(None, 1), dtype=int64)
Tensor("add_18:0", shape=(None, 1), dtype=int64)
507
tipo de aoutput_positivon Tensor("add_30:0", shape=(None, 1), dtype=int64)
Tensor("add_30:0", shape=(None, 1), dtype=int64)
2028
Tensor("ArgMax:0", dtype=int64)
Tensor("strided_slice_2:0", shape=(None,), dtype=int64)
Tensor("GatherV2:0", dtype=int64)
Tensor("strided_slice_3:0", shape=(None,), dtype=int64)
Tensor("GatherV2_1:0", dtype=int64)
tipo de aoutput_positivon Tensor("add_18:0", dtype=int64)
Tensor("add_18:0", dtype=int64)
507
tipo de aoutput_positivon Tensor("add_30:0", dtype=int64)
Tensor("add_30:0

### Loss Function

In [8]:
from tensorflow.keras.losses import Loss,BinaryCrossentropy,MeanSquaredError,MeanSquaredLogarithmicError

def loss_xy(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y




def loss_wh(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h

def loss_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   
    
    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_obj

def loss_no_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    
    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_noobj

## Training with Model.fit()

In [9]:
#anchors =tf.constant(np.array([[0.026,0.062],[0.067,0.183],[0.128,0.323],[0.343,0.650]]),dtype=tf.float32)
anchors =tf.constant(np.array([[0,0,0.02078,0.049],[0,0,0.0426,0.128],[0,0,0.08523,0.19356],[0,0,0.1506,0.4163],[0,0,0.27835,0.58651],[0,0,0.5632,0.78614]]),dtype=tf.float32)

model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "transfer")
model.build(batch_input_shape=(None,416,416,3))
print(model.load_weights_darknet(root_path+"/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Modo entrenamiento
Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
496
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
5232
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
23920
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
98160
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
394096
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
1575792
Pesos de la convolucion (4718592,)
CONV SHAPE (1024, 512, 3, 3)
6298480
Pesos de la convolucion

In [10]:
model.summary()

Model: "tiny_yol_ov3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BasicBlock1 (BasicBlock)     multiple                  496       
_________________________________________________________________
BasicBlock2 (BasicBlock)     multiple                  4736      
_________________________________________________________________
BasicBlock3 (BasicBlock)     multiple                  18688     
_________________________________________________________________
BasicBlock4 (BasicBlock)     multiple                  74240     
_________________________________________________________________
BasicBlock5 (BasicBlock)     multiple                  295936    
_________________________________________________________________
BasicBlock6 (BasicBlock)     multiple                  1181696   
_________________________________________________________________
BasicBlock7 (BasicBlock)     multiple                 

In [11]:
for l in model.layers:
    print(l.name, l.trainable)#,l.weights[0].shape)
    pass

BasicBlock1 False
BasicBlock2 False
BasicBlock3 False
BasicBlock4 False
BasicBlock5 False
BasicBlock6 False
BasicBlock7 False
BasicBlock8 False
BasicBlock9 False
FinalBlock1 True
BasicBlock11 False
BasicBlock12 False
FinalBlock2 True
Concatenate True
Upsampling True
Prediction1 True
Prediction2 True
Concatenate_BBOX True


In [12]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=1e-4)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(0.5),Recall(0.5),TrueNegatives(0.5),TruePositives(0.5),FalseNegatives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[5,5,2,1])
os.chdir(root_path+"/pedestrian_dataset_train_tfr")

In [13]:
history = model.fit(train_dataset, epochs=20,validation_data=val_dataset)

Epoch 1/20
Modo entrenamiento
Modo entrenamiento
   4606/Unknown - 249s 54ms/step - loss: 188.5045 - output_1_loss: 15.8698 - output_2_loss: 5.1983 - output_3_loss: 0.1528 - output_4_loss: 82.8541 - output_3_precision: 0.0022 - output_3_recall: 0.0101 - output_3_true_negatives: 185315600.0000 - output_3_true_positives: 2729.0000 - output_3_false_negatives: 267401.0000 - output_3_false_positives: 1233474.0000 237s 54ms/step - loss:Modo entrenamiento
4607/4607 [==============================] - 256s 56ms/step - loss: 188.5001 - output_1_loss: 15.8694 - output_2_loss: 5.1981 - output_3_loss: 0.1528 - output_4_loss: 82.8522 - output_3_precision: 0.0022 - output_3_recall: 0.0101 - output_3_true_negatives: 185320672.0000 - output_3_true_positives: 2729.0000 - output_3_false_negatives: 267407.0000 - output_3_false_positives: 1233474.0000 - val_loss: 70.1485 - val_output_1_loss: 7.9725 - val_output_2_loss: 1.6891 - val_output_3_loss: 0.0625 - val_output_4_loss: 21.7123 - val_output_3_precision

4607/4607 [==============================] - 267s 58ms/step - loss: 56.4100 - output_1_loss: 6.0075 - output_2_loss: 2.3149 - output_3_loss: 0.0424 - output_4_loss: 14.7124 - output_3_precision: 0.5800 - output_3_recall: 0.0884 - output_3_true_negatives: 186537120.0000 - output_3_true_positives: 23886.0000 - output_3_false_negatives: 246369.0000 - output_3_false_positives: 17299.0000 - val_loss: 47.5191 - val_output_1_loss: 5.7479 - val_output_2_loss: 0.9107 - val_output_3_loss: 0.0424 - val_output_4_loss: 14.1413 - val_output_3_precision: 0.7286 - val_output_3_recall: 0.0786 - val_output_3_true_negatives: 7592830.0000 - val_output_3_true_positives: 929.0000 - val_output_3_false_negatives: 10895.0000 - val_output_3_false_positives: 346.0000
Epoch 12/20
4607/4607 [==============================] - 268s 58ms/step - loss: 55.8064 - output_1_loss: 5.9734 - output_2_loss: 2.2266 - output_3_loss: 0.0424 - output_4_loss: 14.7221 - output_3_precision: 0.5778 - output_3_recall: 0.0876 - output_

4607/4607 [==============================] - 265s 57ms/step - loss: 56.1325 - output_1_loss: 5.9083 - output_2_loss: 2.3613 - output_3_loss: 0.0424 - output_4_loss: 14.6994 - output_3_precision: 0.5793 - output_3_recall: 0.0883 - output_3_true_negatives: 186536960.0000 - output_3_true_positives: 23859.0000 - output_3_false_negatives: 246347.0000 - output_3_false_positives: 17330.0000 - val_loss: 47.2971 - val_output_1_loss: 5.6846 - val_output_2_loss: 0.9269 - val_output_3_loss: 0.0424 - val_output_4_loss: 14.1565 - val_output_3_precision: 0.7299 - val_output_3_recall: 0.0738 - val_output_3_true_negatives: 7592853.0000 - val_output_3_true_positives: 873.0000 - val_output_3_false_negatives: 10951.0000 - val_output_3_false_positives: 323.0000


In [14]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_5521_20_epoch_nadam_0dot00001_mse_3anchors.json', 'w') as fp:
    json.dump(str(history.history), fp)

FileNotFoundError: [Errno 2] No such file or directory: '/home/sergio/Documents/json_experiments'

Se guardo como Fine tunning pero es TRansfer learning 

In [17]:
model.save_weights(root_path+'/weights_saved/pesos_transfer_learning_5521_20_epoch_nadam_0dot00001_mse_3anchors_con_data_aug')